# This is were I tested my teammates queries

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import re,os
import nltk
import pickle
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
ps = PorterStemmer()
import numpy as np
from operator import itemgetter

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Load final index, term_freq, and ai_index

In [3]:
#Merged_block = open("Final_index")
Merged_block = pickle.load(open("final_index_concordia",mode='rb'))

#Term frequency
term_freq = pickle.load(open("term_freq_concordia",mode="rb"))

#ai index
ai_index = pickle.load(open("ai_index_df",mode="rb"))

In [4]:
"Another approach to get length of a document.. Counts the number of times a token occured in some document(its length)."
def get_doc_length(doc_id):
    
    os.chdir("documents-final")
    f = open(str(doc_id)+".txt","r",encoding="utf=8")
    text= f.read()
    os.chdir("..")
    
    return len(text.split(" "))

"Calculates the average length of all documents using the get_doc_length method."
def get_collection_length():
    
    total = 0
    
    os.chdir("documents-final")
    docs = os.listdir()
    os.chdir("..")
    
    for doc in docs:
        if doc.split(".")[0].isdigit():
            total+=get_doc_length(doc)
    
    
        
    return total/17393 # Calibrate number of files
        

"BM25 Ranking formula implementation"
def rank_doc(doc_id,term):
        
    N = 17393 # Total documents number
    k1 = 0.5 # tuning parameter
    b = 0.5 # tuning parameter
    postings_list = []
    
    avg_doc_length = 200.04749944045162


    doc_length = get_doc_length(doc_id)
        
    
    "Find term frequency"
    
    term_frequency = term_freq[term]
    
    "Find document frequency"
    """if term in ai_index.keys():
        document_frequency = ai_index[term]
    else:"""
    document_frequency = len(Merged_block[term])
    
    "Formula steps:"
    
    idf = np.log(N/document_frequency)
    numerator = (k1+1)*term_frequency
    denomenator = k1*((1-b)+ (b*(doc_length/avg_doc_length))) + term_frequency
    
    ratio = numerator/denomenator
    
    weight = idf * ratio
    
    return weight

def rank_doc_tfidf(doc_id,term):
    
    N = 17393 # Total documents number
    postings_list = []
    
    avg_doc_length = 200.04749944045162

    doc_length = get_doc_length(doc_id)
        
    
    "Find term frequency"
    
    term_frequency = term_freq[term]
    
    "Find document frequency"
    if term in ai_index.keys():
        document_frequency = ai_index[term]
    else:
        document_frequency = len(Merged_block[term])
            
    
     
    if document_frequency == 0:
        #"term doesn't exist in document"
        return 0
    
    "Formula steps:"
    
    idf = np.log(N/document_frequency)
    tf = 1 + np.log(term_frequency)
    weight = idf*tf
    
    
    return weight


In [5]:
"This function can run a single query, multi-term AND query, and a multi-term AND query with 1 OR."

def retrieve_docs(query,OR=False,vector_norm=False,tfidf=False,num_ret=10):
    
    terms = query.split(" ")
    temp_postings = []
    postings_list = []
    "Run the following if its a single term query"
    
    if len(terms) == 1:
            
        if terms[0] in Merged_block.keys():
            documents = set(Merged_block[terms[0]])
        else:
            print("Term doesn't exist in the IR system.")
            return
        
        "To store ranked documents"
        ranked_dict = {}
        
        "Rank documents and store document-rankingweight mappings"
        
        for document in documents:
            if tfidf == False:
                ranked_dict[document] = rank_doc(document,terms[0])
            else:
                ranked_dict[document] = rank_doc_tfidf(document,terms[0])


        
        "Sort ranked results"
        ranked_dict = sorted(ranked_dict.items(), key=itemgetter(1),reverse=True)
        
        return ranked_dict
        
    "Retrieve the postings of that term"
    for term in terms:
        if (term) in Merged_block.keys():
            postings_list = list(set(Merged_block[(term)]))
                
           
        if len(postings_list) == 0:
            print("Term not found in IR system, and thus will be ignored.")
            postings_list = []
            continue
        temp_postings.append(postings_list)    
        
        
    "Optimizes querying by dealing with the shortest postings lists first"    
    temp_postings.sort(key=len)
    
    documents = set()
    
    "AND + OR logic"
    for i in range(len(temp_postings)):
        if i == 0:
            documents = documents.union(set(temp_postings[i]))
        if i == 1 and OR == True:
            documents = documents.union(set(temp_postings[i]))
        else:
            documents = documents.intersection(set(temp_postings[i]))
     
    "Store document-rankingWeight mappings"
    ranked_dict = {}
    "Accumaltor for all the query terms in a doc as per the BM25 formula"
    total_rank = 0
    
    for document in documents:
        for term in terms:
            if tfidf == False:
                total_rank+= rank_doc(document,term)
            else:
                total_rank+= rank_doc_tfidf(document,term)


        ranked_dict[document] = total_rank
        
    "Sort ranked results"       
    ranked_dict = sorted(ranked_dict.items(), key=itemgetter(1),reverse=True)
    
    #return list(temp_dict.keys())[:10]
    return ranked_dict[:num_ret]


# Which departments have AI research

In [7]:
query1= "ai research"
retrieve_docs((query1),tfidf=False,num_ret=100)

[(25599, 933.1206311252513),
 (39420, 925.9947634152785),
 (11249, 918.8599598915756),
 (29169, 911.7271850422543),
 (8678, 904.6009531433116),
 (1480, 897.4768989529143),
 (14787, 890.3465971685182),
 (21435, 883.2141332573314),
 (13243, 876.0815341597425),
 (13239, 868.9696731742362),
 (33206, 861.8447427033427),
 (949, 854.7128870530869),
 (6067, 847.5846782942816),
 (11699, 840.4489884714601),
 (17840, 833.3141105360928),
 (13230, 826.1831332600416),
 (25004, 819.0522100291078),
 (36265, 811.919218855358),
 (1958, 804.7833666278143),
 (38306, 797.6485698687137),
 (27547, 790.5186057712046),
 (39830, 783.3847018013383),
 (18322, 776.2531502337679),
 (16269, 769.1172777049954),
 (12171, 761.983177615561),
 (18309, 754.8519436114607),
 (12164, 747.7204123149368),
 (22401, 740.5976046826695),
 (13691, 733.4663166276657),
 (11130, 726.3354136633916),
 (6514, 719.2020438831113),
 (2418, 712.3472098606852),
 (10093, 705.2164622694343),
 (18793, 698.1124227736816),
 (13147, 690.98459194888

In [8]:
query2= "artificial intelligence research"
retrieve_docs((query1),tfidf=False,num_ret=100)

[(25599, 933.1206311252513),
 (39420, 925.9947634152785),
 (11249, 918.8599598915756),
 (29169, 911.7271850422543),
 (8678, 904.6009531433116),
 (1480, 897.4768989529143),
 (14787, 890.3465971685182),
 (21435, 883.2141332573314),
 (13243, 876.0815341597425),
 (13239, 868.9696731742362),
 (33206, 861.8447427033427),
 (949, 854.7128870530869),
 (6067, 847.5846782942816),
 (11699, 840.4489884714601),
 (17840, 833.3141105360928),
 (13230, 826.1831332600416),
 (25004, 819.0522100291078),
 (36265, 811.919218855358),
 (1958, 804.7833666278143),
 (38306, 797.6485698687137),
 (27547, 790.5186057712046),
 (39830, 783.3847018013383),
 (18322, 776.2531502337679),
 (16269, 769.1172777049954),
 (12171, 761.983177615561),
 (18309, 754.8519436114607),
 (12164, 747.7204123149368),
 (22401, 740.5976046826695),
 (13691, 733.4663166276657),
 (11130, 726.3354136633916),
 (6514, 719.2020438831113),
 (2418, 712.3472098606852),
 (10093, 705.2164622694343),
 (18793, 698.1124227736816),
 (13147, 690.98459194888

In [13]:
query3= "artificial intelligence department"
retrieve_docs((query1),tfidf=False,num_ret=100)

[(25599, 933.1206311252513),
 (39420, 925.9947634152785),
 (11249, 918.8599598915756),
 (29169, 911.7271850422543),
 (8678, 904.6009531433116),
 (1480, 897.4768989529143),
 (14787, 890.3465971685182),
 (21435, 883.2141332573314),
 (13243, 876.0815341597425),
 (13239, 868.9696731742362),
 (33206, 861.8447427033427),
 (949, 854.7128870530869),
 (6067, 847.5846782942816),
 (11699, 840.4489884714601),
 (17840, 833.3141105360928),
 (13230, 826.1831332600416),
 (25004, 819.0522100291078),
 (36265, 811.919218855358),
 (1958, 804.7833666278143),
 (38306, 797.6485698687137),
 (27547, 790.5186057712046),
 (39830, 783.3847018013383),
 (18322, 776.2531502337679),
 (16269, 769.1172777049954),
 (12171, 761.983177615561),
 (18309, 754.8519436114607),
 (12164, 747.7204123149368),
 (22401, 740.5976046826695),
 (13691, 733.4663166276657),
 (11130, 726.3354136633916),
 (6514, 719.2020438831113),
 (2418, 712.3472098606852),
 (10093, 705.2164622694343),
 (18793, 698.1124227736816),
 (13147, 690.98459194888

# Which researchers are working on AI

In [20]:
query1= "artificial intelligence department research"
retrieve_docs((query1),tfidf=False,num_ret=100)

[(19951, 1182.9484574718722),
 (29676, 1168.493743722923),
 (21484, 1153.9985201170155),
 (10722, 1139.514845229062),
 (13781, 1125.0443049041944),
 (11725, 1110.5487129836854),
 (1480, 1096.0509516092354),
 (11717, 1081.5544183305235),
 (14787, 1067.0808010783935),
 (21435, 1052.5777148504537),
 (26036, 1038.074218809255),
 (16819, 1023.5971354443718),
 (25004, 1009.0880834852067),
 (13739, 994.5833988164901),
 (15786, 980.4001626197922),
 (21926, 965.9212433861235),
 (24993, 951.423318245281),
 (4511, 936.9291380662197),
 (33691, 922.7620863139878),
 (6033, 908.5788501172899),
 (28558, 894.0959919374303),
 (18309, 879.5906103701441),
 (29057, 865.0903510391851),
 (11130, 850.6063698550377),
 (12663, 836.1005373073217),
 (6514, 821.6085456486461),
 (16239, 807.9250205447889),
 (10093, 793.4580237243367),
 (26474, 779.0408658765978),
 (13671, 764.542776964587),
 (8031, 750.0410633932887),
 (13662, 735.573964779266),
 (13138, 721.0827910399972),
 (6481, 706.6018312469256),
 (34637, 692.

In [19]:
query2= "artificial intelligence paper"
retrieve_docs((query2),tfidf=False,num_ret=100)

[(32510, 528.9932126561899),
 (26871, 513.2533815916199),
 (6514, 498.4255216588706),
 (12018, 483.5979755233708),
 (14832, 468.41751635620267),
 (16239, 452.6774819102669),
 (17512, 436.98213098581823),
 (34918, 421.2627416849458),
 (14947, 406.43517593757815),
 (10722, 390.77599808985144),
 (8031, 375.0767404264509),
 (12245, 359.3812772716763),
 (13138, 343.68276071936236),
 (20686, 327.97201642514744),
 (34637, 312.26763608180295),
 (29253, 296.83021471122095),
 (14008, 281.44820743401726),
 (18232, 265.70986764166065),
 (15923, 250.05303328821424),
 (13739, 234.68109526804386),
 (16169, 219.29910938590086),
 (21926, 203.55561649927262),
 (32290, 187.8261721579089),
 (24993, 172.1102962562992),
 (4511, 156.3849794895619),
 (24475, 141.0208978588098),
 (33691, 125.28106679423986),
 (27928, 109.89908091209688),
 (11286, 94.23485613920775),
 (10259, 78.50272636244874),
 (8067, 62.88689050312721),
 (17411, 47.15852905378574),
 (29057, 31.42484167587819),
 (6401, 15.710766792557436)]

In [21]:
query3= "artificial intelligence topic"
retrieve_docs((query3),tfidf=False,num_ret=100)

[(26871, 554.7565958424043),
 (11126, 540.1316179512429),
 (24308, 524.6298253338259),
 (6514, 509.0809531306652),
 (12018, 494.4562948682714),
 (22513, 479.47219571167244),
 (14576, 463.92203595433443),
 (13671, 448.37541638074435),
 (34918, 432.82806133722204),
 (31844, 418.20338309849774),
 (32471, 402.65276344091683),
 (13781, 387.0964844488533),
 (24144, 371.55599833307417),
 (34637, 356.058792983866),
 (14787, 340.81301996523814),
 (18754, 325.26412477219),
 (13889, 309.71437889830935),
 (15161, 294.16798914265405),
 (14008, 278.62228879389784),
 (33079, 263.07012824240167),
 (15923, 247.67968650354717),
 (37934, 232.50059403152554),
 (33326, 216.97749020362522),
 (35373, 201.4286180004645),
 (15786, 185.9173671293562),
 (15657, 170.39559150250872),
 (24865, 154.93093204334278),
 (32290, 139.38497892022582),
 (12321, 123.85568864360447),
 (31262, 108.31944903745035),
 (1304, 92.7726685866541),
 (22294, 77.23170026583696),
 (11286, 61.74978604877865),
 (1288, 46.20394782427092),
 

# What AI research is being conducted at Concordia

In [22]:
query1= "machine learning"
retrieve_docs((query1),tfidf=False,num_ret=100)

[(14847, 1593.2135907264612),
 (33790, 1584.6746286878238),
 (23549, 1576.1659031579059),
 (21495, 1567.6335867905846),
 (29169, 1559.0950272764946),
 (14832, 1550.5698700126432),
 (27630, 1542.032665993273),
 (17900, 1533.50033905479),
 (21484, 1525.007156020474),
 (26082, 1516.4822729710968),
 (10722, 1507.9520806884088),
 (19423, 1499.4339821145556),
 (32731, 1490.988131466579),
 (30683, 1482.4546837840255),
 (23514, 1473.9212255265188),
 (12245, 1465.3826236438404),
 (13781, 1456.86487232834),
 (23504, 1448.333834810186),
 (1484, 1439.800249650477),
 (3527, 1431.2683983792067),
 (17351, 1422.7363462794174),
 (13251, 1414.206122304487),
 (14787, 1405.6692570678788),
 (19906, 1397.1343497066673),
 (22463, 1388.6217501487072),
 (31675, 1380.0927936151052),
 (29627, 1371.553238288854),
 (21435, 1363.0732347027724),
 (949, 1354.5381157379),
 (26036, 1346.0098666374772),
 (13230, 1337.4883881873632),
 (37802, 1328.9558921054743),
 (27547, 1320.4245587013856),
 (39830, 1311.887397031849),

In [23]:
query2= "deep learning"
retrieve_docs((query2),tfidf=False,num_ret=100)

[(1023, 1320.9908285716328),
 (17917, 1312.6134218769778),
 (12794, 1304.2457084063433),
 (505, 1295.9035242100495),
 (23021, 1287.5223387148121),
 (17897, 1279.144437610559),
 (11240, 1270.7775590092265),
 (15331, 1262.3974320563239),
 (26082, 1254.0191738279582),
 (23514, 1245.6474881317301),
 (26072, 1237.264886082278),
 (37334, 1228.889071932931),
 (11726, 1220.515110116168),
 (14284, 1212.1448769455578),
 (3527, 1203.7700096864626),
 (17349, 1195.3959106989837),
 (13251, 1187.0165258785032),
 (14787, 1178.6361789993782),
 (32704, 1170.2583740366856),
 (1984, 1161.9143570224057),
 (22463, 1153.555404952186),
 (954, 1145.1853224751858),
 (949, 1136.8210701460498),
 (19380, 1128.4519053670394),
 (20399, 1120.0774222682771),
 (35246, 1111.7462926020216),
 (31656, 1103.366509251685),
 (27048, 1094.995179900278),
 (36261, 1086.6197500321782),
 (24993, 1078.2603065789958),
 (21408, 1069.8884701087843),
 (15768, 1061.5131500263128),
 (35222, 1053.2630414612213),
 (32660, 1044.884824441646

In [24]:
query2= "natural language processing"
retrieve_docs((query2),tfidf=False,num_ret=100)

[(12287, 475.2519729276661),
 (33790, 461.56638721110505),
 (3320, 447.9230149679355),
 (26871, 434.24319588958394),
 (6514, 421.1408490024771),
 (12018, 408.0387043466758),
 (13426, 394.71044367491714),
 (11502, 381.02690809633606),
 (34155, 367.3417748923271),
 (3946, 353.6588612675242),
 (38377, 340.07211395853756),
 (34918, 326.4337056863968),
 (1492, 313.3315483913739),
 (11987, 299.6539890250343),
 (1484, 286.0374143513308),
 (11848, 272.36314396492537),
 (17095, 258.686431741763),
 (17732, 245.12797710467652),
 (29760, 231.57404842645778),
 (20923, 217.89209585916635),
 (23356, 204.3413936824297),
 (1083, 190.65785810384855),
 (33079, 177.0052887838742),
 (949, 163.42451998349458),
 (31283, 149.7550593980122),
 (15923, 136.08552831918925),
 (25133, 122.63568632620911),
 (14967, 108.96140182837966),
 (11042, 95.31927802869005),
 (2718, 81.74439785932036),
 (27799, 68.06390067908643),
 (6033, 54.38989840258337),
 (23820, 40.72549678357824),
 (18950, 27.05936241488738),
 (25349, 13